In [49]:
import sentence_transformers
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
import os
import pathlib
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import torch
from transformers import BertConfig, BertForPreTraining, BertTokenizer,BertModel
import random
import logging
import pathlib, os
import nltk
from beir import util, LoggingHandler


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [50]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/cgrdj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data

In [51]:
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

Dataset downloaded here: /Users/cgrdj/Documents/Code/sentence-transformers/datasets/scifact


In [52]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="train") # or split = "train" or "dev"
unsupervised_train_data =  list(queries.values())#+[data['title']+' \n '+data['text'] for data in list(corpus.values())]
random.Random(0).shuffle(unsupervised_train_data)


2024-03-16 00:29:12 - Loading Corpus...


100%|██████████| 5183/5183 [00:00<00:00, 9696.75it/s] 


2024-03-16 00:29:12 - Loaded 5183 TRAIN Documents.
2024-03-16 00:29:12 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 ver

In [53]:
data_path = "datasets/scifact"
test_corpus, test_queries, test_qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

2024-03-16 00:29:14 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

100%|██████████| 5183/5183 [00:00<00:00, 20530.55it/s]


2024-03-16 00:29:14 - Loaded 5183 TEST Documents.
2024-03-16 00:29:14 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

# Model

In [55]:

# Define your sentence transformer model using CLS pooling
model_name = "huawei-noah/TinyBERT_General_4L_312D"
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "cls")
model = SentenceTransformer(modules=[word_embedding_model, pooling_model],device='cuda')




AssertionError: Torch not compiled with CUDA enabled

In [58]:
import torch

In [59]:
torch.cuda.is_available()


False

# TSDAE training

In [42]:
# Define a list with sentences (1k - 100k sentences)
train_sentences = unsupervised_train_data

# Create the special 
# denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(
    model, decoder_name_or_path=model_name, tie_encoder_decoder=True
)


2024-03-10 14:41:55 - When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.


Some weights of BertLMHeadModel were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self

In [43]:

from sentence_transformers import losses, util, models
from sentence_transformers.evaluation import SentenceEvaluator

class SaveCallback(SentenceEvaluator):
    def __init__(self, save_path):
        self.save_path = save_path

    def __call__(self, model, epoch, steps,output_path=None,):
        model.save(f"{self.save_path}/epoch-{epoch}-steps-{steps}")
        return 0

# Initialize the callback
save_callback = SaveCallback(save_path="output/tsdae-model")

In [45]:
total_steps = len(train_dataloader) * 1  # for one epoch, adjust as needed

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=20,
    weight_decay=0.01,
    scheduler="constantlr",
    optimizer_params={"lr": 9e-5},
    show_progress_bar=True,
    evaluator=save_callback,  # Add your save callback here
    evaluation_steps=total_steps,  # Set this to a high number to avoid frequent evaluations, or adjust as necessary
    output_path="output/tsdae-model_queries"  # This ensures the final model is saved at the specified location.
)


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

                                             
                                                          
Epoch:   0%|          | 0/20 [00:27<?, ?it/s]

2024-03-10 14:42:57 - Save model to output/tsdae-model/epoch-0-steps-51
2024-03-10 14:42:58 - Save model to output/tsdae-model_queries
2024-03-10 14:42:58 - Save model to output/tsdae-model/epoch-0-steps--1


                                                     
Epoch:   5%|▌         | 1/20 [00:56<08:36, 27.19s/it]

2024-03-10 14:43:26 - Save model to output/tsdae-model/epoch-1-steps-51
2024-03-10 14:43:27 - Save model to output/tsdae-model/epoch-1-steps--1


...

2024-03-10 14:50:46 - Save model to output/tsdae-model/epoch-17-steps-51
2024-03-10 14:50:46 - Save model to output/tsdae-model/epoch-17-steps--1


                                                      
Epoch:  95%|█████████▌| 19/20 [08:41<00:27, 27.02s/it]

2024-03-10 14:51:12 - Save model to output/tsdae-model/epoch-18-steps-51
2024-03-10 14:51:12 - Save model to output/tsdae-model/epoch-18-steps--1


                                                      
Epoch: 100%|██████████| 20/20 [09:09<00:00, 27.46s/it]

2024-03-10 14:51:39 - Save model to output/tsdae-model/epoch-19-steps-51
2024-03-10 14:51:40 - Save model to output/tsdae-model/epoch-19-steps--1


In [ ]:
loaded_model = SentenceTransformer("output/tsdae-model")


2024-03-09 18:46:41 - Load pretrained SentenceTransformer: output/tsdae-model
2024-03-09 18:46:41 - Use pytorch device_name: mps


# BEIR retrival framework

In [46]:
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

#### Dense Retrieval using SBERT (Sentence-BERT) ####
#### Provide any pretrained sentence-transformers model
#### The model was fine-tuned using cosine-similarity.
#### Complete list - https://www.sbert.net/docs/pretrained_models.html
l=[]
l_index=[]
for model_name in os.listdir("./output/tsdae-model"):
    model = DRES(models.SentenceBERT("output/tsdae-model/"+model_name), batch_size=8)
    retriever = EvaluateRetrieval(model, score_function="cos_sim")
    #### Retrieve dense results (format of results is identical to qrels)
    results = retriever.retrieve(test_corpus, test_queries)


    logging.info("Retriever evaluation for k in: {}".format(retriever.k_values))

    ndcg, _map, recall, precision = retriever.evaluate(test_qrels, results, retriever.k_values,ignore_identical_ids=False)
    ndcg.update(_map)
    ndcg.update(recall)
    ndcg.update(precision)
    l_index.append(model_name)
    l.append(ndcg)

2024-03-10 14:55:36 - Load pretrained SentenceTransformer: output/tsdae-model/epoch-19-steps-51
2024-03-10 14:55:36 - Use pytorch device_name: mps
2024-03-10 14:55:37 - Encoding Queries...


Batches: 100%|██████████| 38/38 [00:02<00:00, 18.31it/s]


2024-03-10 14:55:39 - Sorting Corpus by document length (Longest first)...
2024-03-10 14:55:39 - Encoding Corpus in batches... Warning: This might take a while!
2024-03-10 14:55:39 - Scoring Function: Cosine Similarity (cos_sim)
2024-03-10 14:55:39 - Encoding Batch 1/1...


Batches: 100%|██████████| 648/648 [00:42<00:00, 15.23it/s]


2024-03-10 14:56:23 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2024-03-10 14:56:24 - 

2024-03-10 14:56:24 - NDCG@1: 0.0067
2024-03-10 14:56:24 - NDCG@3: 0.0067
2024-03-10 14:56:24 - NDCG@5: 0.0067
2024-03-10 14:56:24 - NDCG@10: 0.0078
2024-03-10 14:56:24 - NDCG@100: 0.0140
2024-03-10 14:56:24 - NDCG@1000: 0.0582
2024-03-10 14:56:24 - 

2024-03-10 14:56:24 - MAP@1: 0.0067
2024-03-10 14:56:24 - MAP@3: 0.0067
2024-03-10 14:56:24 - MAP@5: 0.0067
2024-03-10 14:56:24 - MAP@10: 0.0071
2024-03-10 14:56:24 - MAP@100: 0.0079
2024-03-10 14:56:24 - MAP@1000: 0.0089
2024-03-10 14:56:24 - 

2024-03-10 14:56:24 - Recall@1: 0.0067
2024-03-10 14:56:24 - Recall@3: 0.0067
2024-03-10 14:56:24 - Recall@5: 0.0067
2024-03-10 14:56:24 - Recall@10: 0.0100
2024-03-10 14:56:24 - Recall@100: 0.0450
2024-03-10 14:56:24 - Recall@1000: 0.4221
2024-03-10 14:56:24 - 

2024-03-10 14:56:24 - P@1: 0.0067
2024-03-10 14:56:24 - P@3: 0.0022
2024-03-10 14:56:24 - P@5: 0.0013
2024-03-10 14:56:24 - P@10: 0.0010

...

2024-03-10 15:25:55 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2024-03-10 15:25:56 - 

2024-03-10 15:25:56 - NDCG@1: 0.0033
2024-03-10 15:25:56 - NDCG@3: 0.0054
2024-03-10 15:25:56 - NDCG@5: 0.0054
2024-03-10 15:25:56 - NDCG@10: 0.0064
2024-03-10 15:25:56 - NDCG@100: 0.0097
2024-03-10 15:25:56 - NDCG@1000: 0.0523
2024-03-10 15:25:56 - 

2024-03-10 15:25:56 - MAP@1: 0.0033
2024-03-10 15:25:56 - MAP@3: 0.0050
2024-03-10 15:25:56 - MAP@5: 0.0050
2024-03-10 15:25:56 - MAP@10: 0.0054
2024-03-10 15:25:56 - MAP@100: 0.0057
2024-03-10 15:25:56 - MAP@1000: 0.0067
2024-03-10 15:25:56 - 

2024-03-10 15:25:56 - Recall@1: 0.0033
2024-03-10 15:25:56 - Recall@3: 0.0067
2024-03-10 15:25:56 - Recall@5: 0.0067
2024-03-10 15:25:56 - Recall@10: 0.0100
2024-03-10 15:25:56 - Recall@100: 0.0283
2024-03-10 15:25:56 - Recall@1000: 0.3894
2024-03-10 15:25:56 - 

2024-03-10 15:25:56 - P@1: 0.0033
2024-03-10 15:25:56 - P@3: 0.0022
2024-03-10 15:25:56 - P@5: 0.0013
2024-03-10 15:25:56 - P@10: 0.0010

Batches: 100%|██████████| 38/38 [00:02<00:00, 16.81it/s]


2024-03-10 15:25:58 - Sorting Corpus by document length (Longest first)...
2024-03-10 15:25:58 - Encoding Corpus in batches... Warning: This might take a while!
2024-03-10 15:25:58 - Scoring Function: Cosine Similarity (cos_sim)
2024-03-10 15:25:58 - Encoding Batch 1/1...


Batches: 100%|██████████| 648/648 [00:55<00:00, 11.74it/s]


2024-03-10 15:26:56 - Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]
2024-03-10 15:26:56 - 

2024-03-10 15:26:56 - NDCG@1: 0.0167
2024-03-10 15:26:56 - NDCG@3: 0.0230
2024-03-10 15:26:56 - NDCG@5: 0.0255
2024-03-10 15:26:56 - NDCG@10: 0.0415
2024-03-10 15:26:56 - NDCG@100: 0.0838
2024-03-10 15:26:56 - NDCG@1000: 0.1526
2024-03-10 15:26:56 - 

2024-03-10 15:26:56 - MAP@1: 0.0167
2024-03-10 15:26:56 - MAP@3: 0.0217
2024-03-10 15:26:56 - MAP@5: 0.0230
2024-03-10 15:26:56 - MAP@10: 0.0291
2024-03-10 15:26:56 - MAP@100: 0.0359
2024-03-10 15:26:56 - MAP@1000: 0.0382
2024-03-10 15:26:56 - 

2024-03-10 15:26:56 - Recall@1: 0.0167
2024-03-10 15:26:56 - Recall@3: 0.0267
2024-03-10 15:26:56 - Recall@5: 0.0333
2024-03-10 15:26:56 - Recall@10: 0.0826
2024-03-10 15:26:56 - Recall@100: 0.2983
2024-03-10 15:26:56 - Recall@1000: 0.8473
2024-03-10 15:26:56 - 

2024-03-10 15:26:56 - P@1: 0.0167
2024-03-10 15:26:56 - P@3: 0.0089
2024-03-10 15:26:56 - P@5: 0.0067
2024-03-10 15:26:56 - P@10: 0.0093

In [48]:
import pandas as pd
pd.DataFrame(l, index=l_index).sort_values('NDCG@1000', ascending=False)


,NDCG@1,NDCG@3,NDCG@5,NDCG@10,NDCG@100,NDCG@1000,MAP@1,MAP@3,MAP@5,MAP@10,...,Recall@5,Recall@10,Recall@100,Recall@1000,P@1,P@3,P@5,P@10,P@100,P@1000
epoch-12-steps-375,0.03667,0.06550,0.07855,0.09680,0.17077,0.21996,0.03667,0.05778,0.06499,0.07211,...,0.11556,0.17233,0.54311,0.93300,0.03667,0.03111,0.02600,0.01967,0.00623,0.00105
epoch-11-steps-375,0.02667,0.04425,0.05710,0.07424,0.13555,0.19423,0.02667,0.03944,0.04625,0.05312,...,0.08639,0.13956,0.44978,0.91567,0.02667,0.02000,0.01933,0.01567,0.00517,0.00103
epoch-10-steps-375,0.02667,0.04513,0.05740,0.07347,0.13412,0.19219,0.02667,0.04056,0.04735,0.05390,...,0.08289,0.13289,0.43811,0.89633,0.02667,0.02000,0.02000,0.01500,0.00510,0.00102
epoch-9-steps-375,0.01667,0.02421,0.03517,0.05226,0.10325,0.16877,0.01667,0.02222,0.02810,0.03509,...,0.05583,0.10706,0.36606,0.88567,0.01667,0.01000,0.01200,0.01200,0.00417,0.00100
epoch-8-steps-375,0.01667,0.02298,0.03425,0.04923,0.09790,0.16222,0.01667,0.02167,0.02743,0.03333,...,0.05361,0.10039,0.35356,0.86733,0.01667,0.00889,0.01200,0.01133,0.00393,0.00098
epoch-7-steps-375,0.01667,0.02298,0.02555,0.04152,0.08380,0.15256,0.01667,0.02167,0.02300,0.02912,...,0.03333,0.08261,0.29828,0.84733,0.01667,0.00889,0.00667,0.00933,0.00330,0.00096
epoch-6-steps-375,0.01333,0.02175,0.02347,0.03917,0.08376,0.14769,0.01333,0.02000,0.02080,0.02724,...,0.03067,0.07844,0.30300,0.81567,0.01333,0.00889,0.00667,0.00867,0.00337,0.00093
epoch-5-steps-375,0.01333,0.01964,0.02281,0.03180,0.07503,0.13839,0.01333,0.01833,0.01997,0.02345,...,0.03067,0.05900,0.27383,0.78522,0.01333,0.00778,0.00667,0.00633,0.00303,0.00089
epoch-4-steps-375,0.01000,0.01421,0.01693,0.02027,0.05438,0.12033,0.01000,0.01333,0.01483,0.01619,...,0.02333,0.03233,0.19856,0.72289,0.01000,0.00556,0.00467,0.00367,0.00220,0.00083
epoch-3-steps-375,0.00333,0.00500,0.00773,0.00969,0.03550,0.10034,0.00333,0.00444,0.00594,0.00665,...,0.01333,0.02000,0.14789,0.67122,0.00333,0.00222,0.00267,0.00200,0.00163,0.00077
